# AlexNet

In [1]:
import tensorflow as tf
import DL_utils as dl

In [2]:
tf.__version__

'1.7.0'

In [3]:
def create_placeholders(n_H, n_W, n_C, n_y):
    '''
    Function to create placeholder for tensorflow session
    
    Args:
    n_H = height of the image
    n_W = width of image
    n_C = number of channels
    n_y = number of output features
    
    returns:
    X,Y
    '''
    X = tf.placeholder(tf.float32, shape = (None, n_H, n_W, n_C))
    Y = tf.placeholder(tf.float32, shape = (None, n_y))
    
    return X, Y

In [4]:
def AN_forward_propagation(X, parameters):
    '''
    Forward propagation for AlexNet without normalization
    
    '''
    # 1st conv
    A1 = dl.conv_layer(X,parameters['W1'],parameters['b1'], [1,4,4,1], padding='VALID', name='1')
    # 1st pool
    P1 = dl.max_pool(A1, kernel=[1,3,3,1], strides=[1,2,2,1], padding = 'VALID', name='1')
    # 2nd conv
    A2 = dl.conv_layer(P1,parameters['W2'],parameters['b2'], strides = [1,1,1,1], padding = 'SAME', name='2')
    #2nd pool
    P2 = dl.max_pool(A2, kernel=[1,3,3,1], strides=[1,2,2,1], padding = 'VALID', name='2')
    # 3rd conv
    A3 = dl.conv_layer(P2,parameters['W3'], parameters['b3'], strides=[1,1,1,1], padding='SAME', name='3')
    # 4th conv
    A4 = dl.conv_layer(A3,parameters['W4'], parameters['b4'], strides=[1,1,1,1], padding = 'SAME', name='4')
    # 5th conv
    A5 = dl.conv_layer(A4,parameters['W5'], parameters['b5'], strides=[1,1,1,1], padding='SAME', name='5')
    # 3rd pool
    P3 = dl.max_pool(A5, kernel=[1,3,3,1], strides =[1,2,2,1], padding = 'VALID', name='3')
    # Flattening the last pooling layer
    P3 = tf.contrib.layers.flatten(P3)
    # FC1 - 4096 neurons
    F1 = dl.fc_layer(P3, 4096, activation_fn=None, name='1')
    # FC2 - 4096 neurons
    F2 = dl.fc_layer(F1, 4096, activation_fn=None, name='2')
    # FC3 - 1000 neurons reping different classes - may be modified for other models
    F3 = dl.fc_layer(F2, 1000, activation_fn=None, name='3')
    
    return F3

In [5]:
def compute_AN_cost(Z, Y):
    '''
    Computes cost 
    
    Args:
    Z8 -- Logits/Linear output from the last fully connected layer
    Y -- labels corresponding to each example in the batch
    
    Returns:
    cost -- cost of the epoch
    '''
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y))
    
    return cost

In [ ]:
def model(xml_path, LOGDIR, MODELDIR, X_test=None, Y_test=None, learning_rate = 0.09, num_epochs = 100, 
          minibatch_size = 64, print_cost = True):
    
    # restting the default graph
    tf.reset_default_graph()
    
    # retrieve image shapes
    #(m, n_Htr, n_Wtr, n_Ctr) = X_train.shape
    #n_Y = Y_train.shape
    
    # global variables
    costs = []
    
    # randomizer
    seed = int(np.random.randint(1,100,1))
    
    # creating placeholders 
    X, Y = create_placeholders(448,448,3,1000)
    
    # initializing parameters
    parameters = initialize_weights(xml_path)
    
    # forward prop
    F3 = AN_forward_propagation(X, parameters)
    
    # compute cost
    with tf.name_scope("cost"):
        cost = compute_AN_cost(F3, Y)
    
    # select the optimizer
    with tf.name_scope("train")
        train_step = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # initialize global variables
    init = tf.global_variables_initializer()
    
    # initialize the session
    with tf.Session() as sess:
        
        # run the initialization for the session
        sess.run(init)
        
        # initializing summaries and file writers
        merged_summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter(LOGDIR)
        writer.add_graph(sess.graph)
        saver = tf.train.Saver()
        
        # for loop for epoch/iterations
        for epoch in range(num_epochs):
            
            # maintain the cost through an epoch
            epoch_cost = 0
            
            # randomizer
            seed += 1
            
            # path to training folder
            PATH = './ImageNet_dataset/training_folder/'
            
            # set up the data
            h5_files = os.listdir(PATH)
            
            # for loop to iterate through the h5 files
            for file in h5_files:
                
                # DIAGNOSTIC print
                print("setting up" + file + "for training")
                
                # open the h5 file to form tensor
                with h5py.File(PATH+file, mode = 'r') as h5_file:
                    
                    # extract features and labels
                    X_train = np.asarray(h5_file['X_train'])
                    Y_train = np.asarray(h5_file['Y_train'])
                    
                    # number of examples
                    (m, n_Htr, n_Wtr, n_Ctr) = h5_file['X_train'].shape
                    
                    # ??? - REASON WHY
                    num_minibatches = int(m/minibatch_size)

                    # generate minibatches
                    minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

                    # iterate through the minibatches
                    for minibatch in minibatches:
                        
                        # procure minibatches
                        (minibatch_X, minibatch_Y) = minibatch
                        # optimize for cost, 
                        _ , minibatch_cost = sess.run([train_step, cost], feed_dict={X: minibatch_X, Y: tf.one_hot(minibatch_Y, 1000)})
                        # cumulative minibatch cost
                        epoch_cost += minibatch_cost/num_minibatches
                        
            if epoch % 1 == 0:
                s = sess.run(merged_summary, feed_dict = {X: X_train, Y: Y_train})
                writer.add_summary(s, epoch)
                save_path = saver.save(sess, MODELDIR)
            
            # Print the cost after every 5 epochs
            if print_cost == True and epoch % 1 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
                # costs.append(epoch_cost)
                
    return pretrain_parameters